In [1]:
# referenced the below site to get started
# https://kaggle2.blob.core.windows.net/forum-message-attachments/118880/4192/main.py 

# import packages

import numpy as np

import os
import sys
import glob
import cv2
import math
import pickle
import datetime
import pandas as pd

from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, ZeroPadding2D


from keras.optimizers import SGD
from keras.utils import np_utils
from keras.models import model_from_json 

from numpy.random import permutation



/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ec2-user/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ec2-user/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Convers

In [2]:
# set random seed and color_type
# color_type = 1 - grayscale
# color_type = 3 - RGB

np.random.seed(2016)
use_cache = 1

color_type_global = 3




In [3]:
# check each image color type; grayscale or rgb and resize

def get_im(path, img_rows, img_cols, color_type=1):
    # Load as grayscale
    if color_type == 1:
        img = cv2.imread(path, 0)
    elif color_type == 3:
        img = cv2.imread(path)
    # Reduce size 224, 224
    resized = cv2.resize(img, (img_cols, img_rows))

    return resized




In [45]:
# add data augmentation
import random
from scipy import ndarray
import skimage as sk
from skimage import transform
from skimage import util
from skimage import io

def img_rotation(image_array: ndarray):
    # pick a random degree of rotation between 25% on the left and 25% on the right
    random_degree = random.uniform(-25, 25)
    return sk.transform.rotate(image_array, random_degree)

def img_noise(image_array: ndarray):
    # add random noise to the image
    return sk.util.random_noise(image_array)

img_rows = 224
img_cols = 224
color_type = 3

for j in range(10):
        print('Augmenting images in folder c{}'.format(j))
        num_files_created = 0
        path = os.path.join('Data', 'imgs', 'train', 'c' + str(j), '*.jpg')
        files = glob.glob(path)
        for fl in files:
            flbase = os.path.basename(fl)
            img = get_im(fl, img_rows, img_cols, color_type)
            rotated_image = img_rotation(img)
            
            save_path = os.path.join('Data', 'imgs', 'train', 'c' + str(j))
            new_file_path = '%s/augmented1_%s' % (save_path, flbase)

            # save rotated image
            io.imsave(new_file_path, rotated_image)
            
            noise_image = img_noise(img)
            
            save_path = os.path.join('Data', 'imgs', 'train', 'c' + str(j))
            new_file_path = '%s/augmented2_%s' % (save_path, flbase)

            # save rotated image
            io.imsave(new_file_path, noise_image)
            
          

Augmenting images in folder c0


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


Augmenting images in folder c1
Augmenting images in folder c2
Augmenting images in folder c3
Augmenting images in folder c4
Augmenting images in folder c5
Augmenting images in folder c6
Augmenting images in folder c7
Augmenting images in folder c8
Augmenting images in folder c9


In [64]:
# add augmented images to driver data csv

import csv
import pandas as pd

df = pd.read_csv('Data/driver_imgs_list.csv', header=0)
df1 = df.copy()
df2 = df.copy()

df1.iloc[:,2] = 'augmented1_'+df1.iloc[:,2]          
df2.iloc[:,2] = 'augmented2_'+df2.iloc[:,2]

merged = df.append([df1,df2])
merged.to_csv('Data/driver_imgs_list_merged.csv', encoding='utf-8', index=False)

print("New driver file saved for augmented images")





New driver file saved for augmented images


In [4]:
# load driver data from local drive
def get_driver_data():
    dr = dict()
    path = os.path.join('Data', 'driver_imgs_list_merged.csv')
    print('Read drivers data')
    f = open(path, 'r')
    line = f.readline()
    while (1):
        line = f.readline()
        if line == '':
            break
        arr = line.strip().split(',')
        dr[arr[2]] = arr[0]
    f.close()
    return dr



In [5]:
# load training data from local drive

def load_train(img_rows, img_cols, color_type=1):
    X_train = []
    y_train = []
    driver_id = []
    driver_data = get_driver_data()

    print('Read train images')
    
    # loop through 'c' folders, images and target data 
    for j in range(10):
        print('Load folder c{}'.format(j))
        path = os.path.join('Data', 'imgs', 'train', 'c' + str(j), '*.jpg')
        files = glob.glob(path)
        for fl in files:
            flbase = os.path.basename(fl)
            img = get_im(fl, img_rows, img_cols, color_type)
            X_train.append(img)
            y_train.append(j)
            driver_id.append(driver_data[flbase])

    unique_drivers = sorted(list(set(driver_id)))
    print('Unique drivers: {}'.format(len(unique_drivers)))
    print(unique_drivers)
    return X_train, y_train, driver_id, unique_drivers



In [6]:
# load test data from local drive

def load_test(img_rows, img_cols, color_type=1):
    print('Read test images')
    path = os.path.join('Data', 'imgs', 'test', '*.jpg')
    files = glob.glob(path)
    X_test = []
    X_test_id = []
    total = 0
    thr = math.floor(len(files)/10)
    
    # loop through test files and print status
    
    for fl in files:
        flbase = os.path.basename(fl)
        img = get_im(fl, img_rows, img_cols, color_type)
        X_test.append(img)
        X_test_id.append(flbase)
        total += 1
        if total % thr == 0:
            print('Read {} images from {}'.format(total, len(files)))

    return X_test, X_test_id



In [7]:
# create pickle file to enable faster processing

def cache_data(data, path):
    print('Path = '+path)
    
    if not os.path.isdir('cache'):
        os.mkdir('cache')
    if os.path.isdir(os.path.dirname(path)):
        
        # modifications for running on Mac OSX
        #file1 = open(path, 'wb')
        #pickle.dump(data, file1)
        #file1.close()
        
        
        max_bytes = 2**31 - 1
        bytes_out = pickle.dumps(data)
        n_bytes = sys.getsizeof(bytes_out)
        with open(path, 'wb') as f_out:
            for idx in range(0, n_bytes, max_bytes):
                f_out.write(bytes_out[idx:idx+max_bytes])
        print('Pickle complete')
    else:
        print('Directory doesnt exists')

# restore data from pickle
        
def restore_data(path):
    data = dict()
    if os.path.isfile(path):
        print('Restore data from pickle........')
        
        # modifications for running on Mac OSX
        #file1 = open(path, 'rb')
        #data = pickle.load(file1) 
        max_bytes = 2**31 - 1
        try:
            input_size = os.path.getsize(path)
            bytes_in = bytearray(0)
            with open(path, 'rb') as f_in:
                for _ in range(0, input_size, max_bytes):
                    bytes_in += f_in.read(max_bytes)
            data = pickle.loads(bytes_in)
            print('Finished loading Pickle data')
        except:
            print('Error loading Pickle data')
            return None

    return data

# save model weights in json

def save_model(model, index, cross=''):
    json_string = model.to_json()
    if not os.path.isdir('cache'):
        os.mkdir('cache')
    json_name = 'architecture' + str(index) + cross + '.json'
    weight_name = 'model_weights' + str(index) + cross + '.h5'
    open(os.path.join('cache', json_name), 'w').write(json_string)
    model.save_weights(os.path.join('cache', weight_name), overwrite=True)
    print('Model weights saved')

# added load_weights paramenter by_name=True referenced link below
# https://github.com/keras-team/keras/pull/8999/commits/fbd106a2dc6cc5fa17c64220d07c7520f7c0b044

def read_model(index, cross=''):
    print('Reading model weights')
    json_name = 'architecture' + str(index) + cross + '.json'
    weight_name = 'model_weights' + str(index) + cross + '.h5'
    model = model_from_json(open(os.path.join('cache', json_name)).read())
    model.load_weights(os.path.join('cache', weight_name), by_name=True)
    return model




In [8]:
# create submisssion file needed for kaggle competition

def create_submission(predictions, test_id, info):
    result1 = pd.DataFrame(predictions, columns=['c0', 'c1', 'c2', 'c3',
                                                 'c4', 'c5', 'c6', 'c7',
                                                 'c8', 'c9'])
    result1.loc[:, 'img'] = pd.Series(test_id, index=result1.index)
    now = datetime.datetime.now()
    if not os.path.isdir('subm'):
        os.mkdir('subm')
    suffix = info + '_' + str(now.strftime("%Y-%m-%d-%H-%M"))
    sub_file = os.path.join('subm', 'submission_' + suffix + '.csv')
    result1.to_csv(sub_file, index=False)




In [9]:
# read train data from local or from pickle and perform transformations
    
def read_and_normalize_and_shuffle_train_data(img_rows, img_cols,
                                              color_type=1):

    cache_path = os.path.join('cache', 'train_r_' + str(img_rows) +
                              '_c_' + str(img_cols) + '_t_' +
                              str(color_type) + '.dat')

    if not os.path.isfile(cache_path) or use_cache == 0:
        train_data, train_target, driver_id, unique_drivers = \
            load_train(img_rows, img_cols, color_type)
        cache_data((train_data, train_target, driver_id, unique_drivers),
                   cache_path)
    else:
        print('Restore train from cache!')
        (train_data, train_target, driver_id, unique_drivers) = \
            restore_data(cache_path)

    train_data = np.array(train_data, dtype=np.uint8)
    train_target = np.array(train_target, dtype=np.uint8)

    #if color_type == 1:
    #   train_data = train_data.reshape(train_data.shape[0], color_type,
    #                                  img_rows, img_cols)
    # modified to channels_last format
    
    if color_type == 1:
        train_data = train_data.reshape(train_data.shape[0],
                                      img_rows, img_cols, color_type)
    #else:
    #   train_data = train_data.transpose((0, 3, 1, 2))
    # modified to channels_last format
    
    else:
        train_data = train_data.transpose((0, 1, 2, 3))

    train_target = np_utils.to_categorical(train_target, 10)
    train_data = train_data.astype('float32')
    mean_pixel = [103.939, 116.779, 123.68]
    for c in range(3):
        train_data[:, c, :, :] = train_data[:, c, :, :] - mean_pixel[c]
    # train_data /= 255
    perm = permutation(len(train_target))
    train_data = train_data[perm]
    train_target = train_target[perm]
    print('Train shape:', train_data.shape)
    print(train_data.shape[0], 'train samples')
    return train_data, train_target, driver_id, unique_drivers



In [10]:
# read and transform test data

def read_and_normalize_test_data(img_rows=224, img_cols=224, color_type=1):
    cache_path = os.path.join('cache', 'test_r_' + str(img_rows) +
                              '_c_' + str(img_cols) + '_t_' +
                              str(color_type) + '.dat')
    if not os.path.isfile(cache_path) or use_cache == 0:
        test_data, test_id = load_test(img_rows, img_cols, color_type)
        cache_data((test_data, test_id), cache_path)
    else:
        print('Restore test from cache!')
        (test_data, test_id) = restore_data(cache_path)

    test_data = np.array(test_data, dtype=np.uint8)
    
    #if color_type == 1:
    #   train_data = train_data.reshape(train_data.shape[0], color_type,
    #                                  img_rows, img_cols)
    # modified to channels_last format
     
    if color_type == 1:
        test_data = test_data.reshape(test_data.shape[0],
                                      img_rows, img_cols, color_type)
    #else:
    #   test_data = test_data.transpose(0, 3, 1, 2))
    # modified to channels_last format
    
    else:
        test_data = test_data.transpose((0, 1, 2, 3))

    test_data = test_data.astype('float32')
    mean_pixel = [103.939, 116.779, 123.68]
    for c in range(3):
        test_data[:, c, :, :] = test_data[:, c, :, :] - mean_pixel[c]
    # test_data /= 255
    print('Test shape:', test_data.shape)
    print(test_data.shape[0], 'test samples')
    return test_data, test_id




In [11]:
# kfolds means config

def merge_several_folds_mean(data, nfolds):
    a = np.array(data[0])
    for i in range(1, nfolds):
        a += np.array(data[i])
    a /= nfolds
    return a.tolist()






In [12]:
# created simplified approach since defined vgg16 model would take 180 hours (3 epochs * 2 folds * 180 = 1080) 
# per epoch on macbook pro cpu. 
# attempted on aws gpu instance, but ran into OOM issues
# referenced the following site for transfer learning techniques 
# https://keras.io/applications/#vgg16

from keras.applications.vgg16 import VGG16    
from keras.models import Model

def vgg16_model(img_rows, img_cols, color_type=1, num_classes=None):
    model = VGG16(weights='imagenet', include_top=True, input_shape=((224, 224,3)))
    model.layers.pop()
    model.outputs = [model.layers[-1].output]
    model.layers[-1].outbound_nodes = []
    x=Dense(num_classes, activation='softmax')(model.output)
    model=Model(model.input,x)

# set first 8 layers to non-trainable

    for layer in model.layers[:8]:
       layer.trainable = False


    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

 


In [13]:
# cross validation, fit model, save model outputs

def run_cross_validation(nfolds=2, nb_epoch=3, split=0.2, modelStr=''):

    # Now it loads color image
    # input image dimensions
    img_rows, img_cols = 224, 224
    # reduced batch size from 64
    batch_size = 128
    random_state = 20

    train_data, train_target, driver_id, unique_drivers = \
        read_and_normalize_and_shuffle_train_data(img_rows, img_cols,
                                                  color_type_global)

    num_fold = 0
    kf = KFold(len(unique_drivers), n_folds=nfolds,
               shuffle=True, random_state=random_state)
    for train_drivers, test_drivers in kf:
        num_fold += 1
        print('Start KFold number {} of {}'.format(num_fold, nfolds))

        # model = vgg_std16_model(img_rows, img_cols, color_type_global)
        model = vgg16_model(img_rows, img_cols, color_type_global, 10)
        
        model.fit(train_data, train_target, batch_size=batch_size,
                  epochs=nb_epoch,
                  verbose=1, 
                  validation_split=split, shuffle=True)



        save_model(model, num_fold, modelStr)

    print('Start testing............')
    test_data, test_id = read_and_normalize_test_data(img_rows, img_cols,
                                                      color_type_global)
    yfull_test = []

    for index in range(1, num_fold + 1):
        # 1,2,3,4,5
        # Store test predictions
        model = read_model(index, modelStr)
        test_prediction = model.predict(test_data, batch_size=128, verbose=1)
        yfull_test.append(test_prediction)

    info_string = 'loss_' + modelStr \
                  + '_r_' + str(img_rows) \
                  + '_c_' + str(img_cols) \
                  + '_folds_' + str(nfolds) \
                  + '_ep_' + str(nb_epoch)

    test_res = merge_several_folds_mean(yfull_test, nfolds)
    create_submission(test_res, test_id, info_string)



In [16]:
# pass in parameters for nfolds, epochs, split, model string

run_cross_validation(2, 10, 0.15, '_vgg_16_2x10')



Restore train from cache!
Restore data from pickle........
Finished loading Pickle data
Train shape: (67272, 224, 224, 3)
67272 train samples
Start KFold number 1 of 2
Train on 57181 samples, validate on 10091 samples
Epoch 1/10
57181/57181 [==============================] - 2987s 52ms/step - loss: 2.2572 - acc: 0.2951 - val_loss: 2.1718 - val_acc: 0.4918
Epoch 2/10
57181/57181 [==============================] - 3049s 53ms/step - loss: 2.0399 - acc: 0.5590 - val_loss: 1.9066 - val_acc: 0.5921
Epoch 3/10
57181/57181 [==============================] - 2989s 52ms/step - loss: 1.7834 - acc: 0.5979 - val_loss: 1.6657 - val_acc: 0.6026
Epoch 4/10
57181/57181 [==============================] - 2955s 52ms/step - loss: 1.5655 - acc: 0.6041 - val_loss: 1.4716 - val_acc: 0.6054
Epoch 5/10
57181/57181 [==============================] - 2979s 52ms/step - loss: 1.3899 - acc: 0.6067 - val_loss: 1.3242 - val_acc: 0.6030
Epoch 6/10
57181/57181 [==============================] - 2985s 52ms/step - loss: 